In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display("text/html", "<style>.container { width:100% !important; }</style>")

In [ ]:
import pickle
import datashader as ds
import holoviews as hv
from holoviews.operation.datashader import datashade, dynspread, spread
from holoviews import opts
from holoviews.selection import link_selections
import panel as pn
hv.extension('bokeh')

from query_darks import counts_by_mjd, all_darks
import within_saa

In [ ]:
#df2 = counts_by_mjd(mjdstart=56500, mjdend=56510)
df = all_darks()
df.sort_values(by=["expstart"], inplace=True)

In [ ]:
flags = within_saa.saaFilter(df["longitude"].to_numpy(), df["latitude"].to_numpy())
df_flag = df.loc[flags]
df_noflag = df.loc[flags == False]

In [ ]:
p = pickle.load(open("stats.p", "rb"))

In [ ]:
import param
import panel as pn

class DarkExplorer(param.Parameterized):
    flagsaa = param.Boolean(default=False)
    region = param.ObjectSelector(default="inner", objects=set(df["region"]))
    segment = param.ObjectSelector(default="FUVA", objects=["FUVA", "FUVB"])
    hv = param.ObjectSelector(default=167, objects=set(df["hv"]))
    
    xaxis = param.ObjectSelector(default="expstart", objects=df.columns)
    yaxis = param.ObjectSelector(default="dark_pha11", objects=df.columns)
    
    @param.depends("region", "segment", "hv", "xaxis", "yaxis")
    def saa_false(self):
        dfplot_noflag = df_noflag.loc[(df_noflag["segment"] == self.segment) & (df_noflag["hv"] == self.hv) & (df_noflag["region"] == self.region)]
        sctr_noflag = hv.Scatter((dfplot_noflag[self.xaxis], dfplot_noflag[self.yaxis]))
        return sctr_noflag
    
    @param.depends("region", "segment", "hv", "xaxis", "yaxis")
    def saa_true(self):
        dfplot_flag = df_flag.loc[(df_flag["segment"] == self.segment) & (df_flag["hv"] == self.hv) & (df_flag["region"] == self.region)]
        sctr_flag = hv.Scatter((dfplot_flag[self.xaxis], dfplot_flag[self.yaxis]))
        return sctr_flag
    
    @param.depends("flagsaa")
    def view(self):
        dmap_noflag = hv.DynamicMap(self.saa_false)
        #dmap_ds = dynspread(datashade(dmap_noflag)).opts(framewise=True, height=700, width=1000)
        dmap_ds = dmap_noflag.opts(framewise=True, height=700, width=1000)
        if self.flagsaa:
            dmap_flag = hv.DynamicMap(self.saa_true)
            #dmap_flag_ds = dynspread(datashade(dmap_flag, cmap=["red"])).opts(framewise=True, height=700, width=1000)
            dmap_flag_ds = dmap_flag.opts(color="red", framewise=True, height=700, width=1000)
            dmap_ds = dmap_ds * dmap_flag_ds
        return dmap_ds
            

In [ ]:
explorer = DarkExplorer()
pn.Row(explorer.param, explorer.view)

In [ ]:
#explorer = DarkExplorer()
#dark_dmap = dynspread(datashade(hv.DynamicMap(explorer.load_data).opts(framewise=True), cmap=["cornflowerblue"]).opts(framewise=True), threshold=0.5).opts(framewise=True, height=700, width=1000)
##dark_dmap = hv.DynamicMap(explorer.load_data)
#pn.Row(pn.panel(explorer.param, parameters=["flagsaa", "region", "segment", "hv", "xaxis", "yaxis"]), dark_dmap)